In [1]:
import pandas as pd
from transformers import AutoTokenizer

In [2]:
# Load the first 10,000 rows from the training dataset

df=pd.read_csv('Train.csv')
df=df[:10000]

In [3]:
df.head()

,Tweet_ID,tweet,type
0,ID_0022DWKP,Had a dream i got raped last night. By a guy i...,sexual_violence
1,ID_00395QYM,he thought the word raped means sex and told m...,sexual_violence
2,ID_003EOSSF,She NOT TALKING TO ME I WAS RAPED BY 2 MEN 1 M...,sexual_violence
3,ID_004BBHOD,I was sexually abused for 3 years at age 4 to ...,sexual_violence
4,ID_004F7516,Chessy Prout can do better by telling the trut...,sexual_violence


In [4]:
# Rename 'type' column to 'labels' and map violence categories to numeric labels
df=df.rename(columns={'type':'labels'})
df['labels'].replace({
    'Harmful_Traditional_practice': 0,
    'Physical_violence': 1,
    'sexual_violence': 2,
    'economic_violence': 3,
    'emotional_violence': 4
}, inplace=True)
df.head()

/tmp/ipython-input-332080494.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['labels'].replace({
/tmp/ipython-input-332080494.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['labels'].replace({


,Tweet_ID,tweet,labels
0,ID_0022DWKP,Had a dream i got raped last night. By a guy i...,2
1,ID_00395QYM,he thought the word raped means sex and told m...,2
2,ID_003EOSSF,She NOT TALKING TO ME I WAS RAPED BY 2 MEN 1 M...,2
3,ID_004BBHOD,I was sexually abused for 3 years at age 4 to ...,2
4,ID_004F7516,Chessy Prout can do better by telling the trut...,2


In [5]:
# Import and initialize the BERT tokenizer (bert-base-uncased)
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test=train_test_split(df['tweet'], df['labels'], test_size=0.2)

In [7]:
# Split the dataset into training and testing sets (80/20 split)
import torch

train_encodings = tokenizer.batch_encode_plus(
    x_train.tolist(), padding=True, truncation=True, return_tensors="pt"
)

test_encodings = tokenizer.batch_encode_plus(
    x_test.tolist(), padding=True, truncation=True, return_tensors="pt"
)

train_encodings["labels"] = torch.tensor(y_train.tolist())
test_encodings["labels"] = torch.tensor(y_test.tolist())

In [8]:
from torch.utils.data import Dataset

class TweetDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings["input_ids"])

# Add labels
train_dataset=TweetDataset(train_encodings)
eval_dataset=TweetDataset(test_encodings)


In [9]:
train_dataset[1]

{'input_ids': tensor([  101,  1996,  5409,  2518,  2000,  2412,  4148,  2000,  2033,  2060,
          2084,  2026, 18224,  6190,  2003,  2242,  2008,  3047,  2000,  2619,
          2842,  1012,  2026,  4470,  2001, 15504,  2011,  1037,  5011,  1998,
          2085,  2026,  3611,  2038, 12084,  8056,  1010,  2002,  1521,  1055,
          2019, 14813,  1998,  2009,  1521,  1055,  2074,  2428, 21746,  2039,
          1012,  2044,  2002,  4152,  2125,  2147,  2035,  2002,  2515,  2003,
          4392,   102,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [ ]:

# Get the model
from transformers import AutoModelForSequenceClassification

model=AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
#Model training arguments
from transformers import TrainingArguments, Trainer
training_args=TrainingArguments(
    output_dir='/training_results',
    eval_strategy='epoch',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    num_train_epochs=7

)

#setting up the trainer
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

#training the model
trainer.train()

In [ ]:
#model prediction
from sklearn.metrics import accuracy_score
preds=trainer.predict(eval_dataset)
predictions=preds.predictions.argmax(axis=1)
labels=preds.label_ids
accuracy = accuracy_score(labels, predictions)
print(accuracy)

In [ ]:
from datasets import Dataset

#load the test dataset
testdf=pd.read_csv('Test.csv')

tokenized_test = tokenizer(
    testdf['tweet'].tolist(),
    padding=True,
    truncation=True,
    return_tensors="pt"
)

#Hugging Face Dataset
hf_test_dataset = Dataset.from_dict({
    'input_ids': tokenized_test['input_ids'],
    'attention_mask': tokenized_test['attention_mask'],
    'Tweet_ID': testdf['Tweet_ID']
})

predictions = trainer.predict(hf_test_dataset)
predicted_labels = predictions.predictions.argmax(axis=1)


# Create a results DataFrame
results_df = pd.DataFrame({
    'Tweet_ID': testdf['Tweet_ID'],
    'Predicted_Label': predicted_labels
})

# Save to a csv
results_df.to_csv('predictions.csv', index=False)
